In [ ]:
from datamodules import LucasDataModule
import model_constructor
import models
from models.lightning_wrappers import ClassificationWrapper
import torch
from omegaconf import OmegaConf
import torchmetrics
import numpy as np
import matplotlib.pyplot as plt
import copy
import importlib
import models.resnet
importlib.reload(models.resnet)
importlib.reload(model_constructor)
importlib.reload(models);
%matplotlib widget

In [ ]:
model_dict = torch.load("./artifacts/model-0genvn31:best/model.ckpt", map_location=torch.device("cpu"))
model_dict.keys()

In [ ]:
ojb = LucasDataModule(data_dir="data", pin_memory=False, end_cutoff_timesteps=8, data_type="sequence")

In [ ]:
ojb.prepare_data(); ojb.setup()

In [ ]:
cfg = OmegaConf.load("cfg/config.yaml")
cfg.device = "cpu"
cfg.dataset.data_type = "sequence"
network = model_constructor.construct_model(cfg, ojb)
network.load_state_dict(model_dict["state_dict"])
network.eval();

In [ ]:
x, y, lens = next(iter(ojb.val_dataloader()))
ind = None
for i, yi in enumerate(y):
    if yi == 1:
        ind = i
        break
y[ind], x[ind].shape, lens[ind]

In [ ]:
x[ind].unsqueeze(0).shape

In [ ]:
sample_x, sample_y = x[ind].to("cpu"), y[ind]
out_y = None
resnet = network.network
with torch.no_grad():
    x_in = (x[ind].unsqueeze(0), torch.tensor([lens[ind]]))
    out_y = resnet.forward_unrolled(x_in).squeeze(0, 1)

out_y.shape

In [ ]:
sample_len = lens[ind]
sample_start = 15
plt.plot(np.arange(sample_start, sample_len), out_y.numpy()[sample_start:sample_len])

In [ ]:
outs = resnet.forward_unrolled((x, lens)).detach().squeeze(1)

In [ ]:
plt.clf()
for i in range(outs.shape[0]):
    s_len = lens[i]
    start = outs.shape[-1] - s_len + 15
    end = outs.shape[-1]
    plt.plot(np.arange(start, end), outs[i].numpy()[15:s_len], c="r" if y[i] == 1 else "g")

In [ ]:
plt.show()